In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print("Данные загружены!")
print(f"Размер учебника (train): {train.shape}")
print(f"Размер экзамена (test): {test.shape}")

Данные загружены!
Размер учебника (train): (700000, 26)
Размер экзамена (test): (300000, 25)


In [3]:
print(train.columns.tolist())

['id', 'age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'gender', 'ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status', 'family_history_diabetes', 'hypertension_history', 'cardiovascular_history', 'diagnosed_diabetes']


In [4]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  700000 non-null  int64  
 1   age                                 700000 non-null  int64  
 2   alcohol_consumption_per_week        700000 non-null  int64  
 3   physical_activity_minutes_per_week  700000 non-null  int64  
 4   diet_score                          700000 non-null  float64
 5   sleep_hours_per_day                 700000 non-null  float64
 6   screen_time_hours_per_day           700000 non-null  float64
 7   bmi                                 700000 non-null  float64
 8   waist_to_hip_ratio                  700000 non-null  float64
 9   systolic_bp                         700000 non-null  int64  
 10  diastolic_bp                        700000 non-null  int64  
 11  heart_rate                

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

target_col = 'diagnosed_diabetes'
X = train.drop(['id', target_col], axis=1)
y = train[target_col]


for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        # И в тесте тоже надо поправить, иначе потом будет ошибка
        if col in test.columns:
            test[col] = test[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("🚀 Обучение (теперь без ошибок)...")
model = lgb.LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
score = roc_auc_score(y_val, probs)
print(f"\n🏆 ROC-AUC: {score:.5f}")

🚀 Обучение (теперь без ошибок)...

🏆 ROC-AUC: 0.72191


In [ ]:
X_test = test.drop(['id'], axis=1)

test_probs = model.predict_proba(X_test)[:, 1]

submission = pd.DataFrame({
    'id': test['id'],
    'diagnosed_diabetes': test_probs
})

submission.to_csv('submission.csv', index=False)

print("Файл submission.csv готов! Проверь папку проекта.")
print(submission.head())

Файл submission.csv готов! Проверь папку проекта.
       id  diagnosed_diabetes
0  700000            0.519037
1  700001            0.670999
2  700002            0.782870
3  700003            0.414762
4  700004            0.886695


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target_col = 'diagnosed_diabetes'

X = train.drop(['id', target_col], axis=1)
y = train[target_col]
X_test = test.drop(['id'], axis=1)

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(
    n_estimators=500,        # Даем ей больше попыток (было 100)
    learning_rate=0.05,      # Учимся медленнее и внимательнее (было ~0.1)
    max_depth=6,             # Ограничиваем глубину (чтобы не зубрила)
    num_leaves=31,           # Стандартное ограничение "листвы"
    reg_lambda=10,           # Жесткий "штраф" за сложность (Regularization)
    random_state=42,
    verbose=-1
)

print("🧠 Начинаю углубленное обучение...")
model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
new_score = roc_auc_score(y_val, probs)

print(f"\nСтарый результат (валидация): ~0.72191")
print(f"🔥 Новый результат (валидация): {new_score:.5f}")

🧠 Начинаю углубленное обучение...

Старый результат (валидация): ~0.72191
🔥 Новый результат (валидация): 0.72536


In [ ]:
tuned_probs = model.predict_proba(X_test)[:, 1]

submission_tuned = pd.DataFrame({
    'id': test['id'],
    'diagnosed_diabetes': tuned_probs
})

submission_tuned.to_csv('submission_tuned.csv', index=False)


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_eng = train.copy()
test_eng = test.copy()

train_eng['Risk_Factor'] = train_eng['age'] * train_eng['bmi']
test_eng['Risk_Factor'] = test_eng['age'] * test_eng['bmi']

train_eng['Waist_Age'] = train_eng['waist_to_hip_ratio'] * train_eng['age']
test_eng['Waist_Age'] = test_eng['waist_to_hip_ratio'] * test_eng['age']

print("🧪 Новые признаки созданы!")

target_col = 'diagnosed_diabetes'
X = train_eng.drop(['id', target_col], axis=1)
y = train_eng[target_col]
X_test = test_eng.drop(['id'], axis=1)

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(
    n_estimators=600,       # Чуть добавим деревьев, т.к. данных стало больше
    learning_rate=0.05,
    max_depth=6,
    num_leaves=31,
    reg_lambda=10,
    random_state=42,
    verbose=-1
)

print("🧠 Обучаю модель на новых данных...")
model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
score = roc_auc_score(y_val, probs)
print(f"\n🚀 Результат с новыми фичами (валидация): {score:.5f}")

# 7. Сохраняем файл для Kaggle
final_probs = model.predict_proba(X_test)[:, 1]
sub = pd.DataFrame({'id': test['id'], 'diagnosed_diabetes': final_probs})
sub.to_csv('submission_feature.csv', index=False)
print("✅ Файл submission_feature.csv готов к отправке!")

🧪 Новые признаки созданы!
🧠 Обучаю модель на новых данных...

🚀 Результат с новыми фичами (валидация): 0.72543
✅ Файл submission_feature.csv готов к отправке!


In [ ]:
X_final = train_eng.copy()
X_test_final = test_eng.copy()

X_final['Cholesterol_Ratio'] = X_final['ldl_cholesterol'] / (X_final['hdl_cholesterol'] + 0.1)
X_test_final['Cholesterol_Ratio'] = X_test_final['ldl_cholesterol'] / (X_test_final['hdl_cholesterol'] + 0.1)

X_final['BP_Diff'] = X_final['systolic_bp'] - X_final['diastolic_bp']
X_test_final['BP_Diff'] = X_test_final['systolic_bp'] - X_test_final['diastolic_bp']

X_final['Total_Lipids'] = X_final['cholesterol_total'] + X_final['triglycerides']
X_test_final['Total_Lipids'] = X_test_final['cholesterol_total'] + X_test_final['triglycerides']

print("💊 Медицинские фичи добавлены!")

target_col = 'diagnosed_diabetes'
X = X_final.drop(['id', target_col], axis=1)
y = X_final[target_col]
X_test_submit = X_test_final.drop(['id'], axis=1)

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        if col in X_test_submit.columns:
            X_test_submit[col] = X_test_submit[col].astype('category')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier(
    n_estimators=700,
    learning_rate=0.04,  # Еще чуть медленнее для точности
    max_depth=7,
    num_leaves=35,
    random_state=42,
    verbose=-1
)

print("🧠 Обучение финальной модели...")
model.fit(X_train, y_train)

probs = model.predict_proba(X_val)[:, 1]
score = roc_auc_score(y_val, probs)
print(f"\n🔥 Результат с медицинскими фичами: {score:.5f}")

final_probs = model.predict_proba(X_test_submit)[:, 1]
sub = pd.DataFrame({'id': test['id'], 'diagnosed_diabetes': final_probs})
sub.to_csv('submission_medical.csv', index=False)
print("✅ Файл submission_medical.csv готов! Это наша лучшая попытка.")

💊 Медицинские фичи добавлены!
🧠 Обучение финальной модели...

🔥 Результат с медицинскими фичами: 0.72530
✅ Файл submission_medical.csv готов! Это наша лучшая попытка.


In [2]:
%pip install catboost

  Using cached catboost-1.2.8-cp312-cp312-win_amd64.whl.metadata (1.5 kB)
  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached catboost-1.2.8-cp312-cp312-win_amd64.whl (102.4 MB)
Using cached graphviz-0.21-py3-none-any.whl (47 kB)
Using cached plotly-6.5.0-py3-none-any.whl (9.9 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['Risk_Factor'] = train['age'] * train['bmi']
test['Risk_Factor'] = test['age'] * test['bmi']

target_col = 'diagnosed_diabetes'
X = train.drop(['id', target_col], axis=1)
y = train[target_col]
X_test = test.drop(['id'], axis=1)

cat_features = [col for col in X.columns if X[col].dtype == 'object']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model_cat = CatBoostClassifier(
    iterations=1000,         # Пусть учится дольше
    learning_rate=0.05,
    depth=6,
    eval_metric='AUC',       # Сразу затачиваем под метрику соревнования
    random_seed=42,
    verbose=100              # Будет писать отчет каждые 100 шагов
)

print("🐱 Выпускаем кота...")
model_cat.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50 # Если 50 шагов нет улучшений - стоп
)

final_probs = model_cat.predict_proba(X_test)[:, 1]
sub = pd.DataFrame({'id': test['id'], 'diagnosed_diabetes': final_probs})
sub.to_csv('submission_catboost.csv', index=False)
print("\n✅ Файл submission_catboost.csv готов!")

🐱 Выпускаем кота...
0:	test: 0.6774204	best: 0.6774204 (0)	total: 562ms	remaining: 9m 21s
100:	test: 0.7045720	best: 0.7045720 (100)	total: 41.9s	remaining: 6m 12s
200:	test: 0.7099146	best: 0.7099146 (200)	total: 1m 22s	remaining: 5m 27s
300:	test: 0.7144516	best: 0.7144516 (300)	total: 2m 1s	remaining: 4m 43s
400:	test: 0.7184306	best: 0.7184306 (400)	total: 2m 41s	remaining: 4m 1s
500:	test: 0.7201967	best: 0.7201967 (500)	total: 3m 21s	remaining: 3m 21s
600:	test: 0.7213692	best: 0.7213692 (600)	total: 4m 2s	remaining: 2m 40s
700:	test: 0.7225456	best: 0.7225458 (699)	total: 4m 42s	remaining: 2m
800:	test: 0.7234827	best: 0.7234827 (800)	total: 5m 25s	remaining: 1m 20s
900:	test: 0.7240176	best: 0.7240176 (900)	total: 6m 5s	remaining: 40.2s
999:	test: 0.7245107	best: 0.7245107 (999)	total: 6m 46s	remaining: 0us

bestTest = 0.7245106595
bestIteration = 999


✅ Файл submission_catboost.csv готов!


In [ ]:
import pandas as pd

sub_lgbm = pd.read_csv('submission_feature.csv')   # Твой рекорд (0.6969)
sub_cat = pd.read_csv('submission_catboost.csv')   # Твой новый Кот (0.6961)

blend_pred = (sub_lgbm['diagnosed_diabetes'] * 0.6) + (sub_cat['diagnosed_diabetes'] * 0.4)

sub_blend = pd.DataFrame({
    'id': sub_lgbm['id'],
    'diagnosed_diabetes': blend_pred
})

sub_blend.to_csv('submission_blend.csv', index=False)

print("🍹 Коктейль готов! Файл submission_blend.csv создан.")
print("Это математически должно дать лучший результат, чем каждая модель по отдельности.")

🍹 Коктейль готов! Файл submission_blend.csv создан.
Это математически должно дать лучший результат, чем каждая модель по отдельности.
